# Word Embedding

자연어를 컴퓨터가 이해하고, 효율적으로 처리하게 하기 위해서는 컴퓨터가 이해할 수 있도록 자연어를 적절히 변환할 필요가 있습니다. 단어를 표현하는 방법에 따라서 자연어 처리의 성능이 크게 달라지기 때문에 이에 대한 많은 연구가 있었고, 여러가지 방법들이 알려져 있습니다.

최근에는 단어의 의미를 벡터화시킬 수 있는 이번 챕터에서 배우게 될 워드투벡터(Word2Vec)와 글로브(Glove)가 많이 사용되고 있습니다. 이번 챕터에서는 전통적 방법의 한계를 개선시킨 워드 임베딩(Word Embedding) 방법론에 대해서 배워보도록 하겠습니다.

# 영어 Word2vec 만들기

영어 데이터를 다운받아 직접 Word2Vec 작업을 해보자

In [2]:
import re #regular expression
from lxml import etree
import nltk 
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
targetXML=open('datasets/ted_en-20160408/ted_en-20160408.xml', 'r', encoding='UTF8')

target_text = etree.parse(targetXML)

parse_text = '\n'.join(target_text.xpath('//content/text()'))

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (audio) (laughter) 등의 배경음 부분 제거
# 해당코드는 괄호로 구성된 나용을 제거 한다

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK 를 이용하여 문장 토큰화를 수행


print(len(sent_text))
normalized_text = []

for string in sent_text :
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
    
result = []

result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화 수행


print(result[:10])

273424
[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doing', 'exactly', 'the', 'same']]


In [5]:
print(len(result))

273424


In [6]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

# size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBOW, 1은 Skip-gram.

In [15]:
# model 의 형태를 보자
print(model)

Word2Vec(vocab=21613, size=100, alpha=0.025)


In [9]:
a = model.wv.most_similar("man")
print(a)

[('woman', 0.8423302173614502), ('guy', 0.8012086749076843), ('lady', 0.789531946182251), ('girl', 0.7496041655540466), ('boy', 0.7466311454772949), ('gentleman', 0.7353699207305908), ('soldier', 0.7269606590270996), ('rabbi', 0.6808528900146484), ('kid', 0.6792615652084351), ('poet', 0.6763109564781189)]


# 글로브(GloVe)

글로브 (Global Vectors for Word Representaion) 는 카운트 기반과 예측 기반을 모두 사용하는 방법의 단어 임베딩 방법론이다.

카운트 기반의 LSA 와 예측기반의 Word2vec 의 단점을 보완 하기 위해 나왔다.

lsa 는 dtm 이나 tf-idf 행렬과 같이 각 문서에서 각 단어의 빈도수를 카운트 한 행렬이라는 전체적인 통계 정보를 입력어로 받아 차원을 축소 (Truncated SVC) 하여 잠재된 의미를 끌어내는 방법이였다.

Word2vec 은 실제 값과 예측값에 대한 오차를 손실 함수를 통해 줄여가며 학습하는 예측기반 방법론이다.

LSA 는 전체적인 통계 정보를 고려하지만 의미 유추 작업에는 성능이 떨어진다.

Word2Vec 는 예측 기반으로 단어 유추작업에는 LSA 보다 뛰어나지만 임베딩 벡터가 윈도우 크기내에서만 주변 단어를 고려하기 떄문에 코퍼스 전체적인 통계 정보를 고려하지 못한다/

GloVe 는 카운트 기반과 예측 기반 방법론 두가지 모두를 사용한다.


GloVe는 '임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것이다' 정확히는 이를 만족하는 임베딩 벡터를 만드는 과정이다.

    
    dot(w_i,w_k) ≈ P(k|i)
    
    w_i : 중심 단어 i의 임베딩 벡터
    w_k : 주변 단어 k의 임베딩 벡터
    Pik  : P(k | i) = XikXi : 중심 단어 i가 등장했을 때 윈도우 내 주변 단어 k가 등장할 확률
    
    dot product(wi wk~)≈ log P(k | i)=log Pik
    

최종 유도 :
- Loss function = 1∑V   f(X_mn)(wT_m * wn + b_m + b_n − log(X_mn))^2

f(X_mn) 는 뭐야??? : 동시 등장 행렬에서 동시 등장 빈도의 값 Xik이 굉장히 낮은 경우에는 정보에 거의 도움이 되지 않는다고 판단합니다. 그래서 이에 대한 가중치를 주는 고민을 하게 되는데 GloVe 연구팀이 선택한 것은 바로 Xik의 값에 영향을 받는 가중치 함수(Weighting function) f(Xik)를 손실 함수에 도입하는 것입니다.

# Using pre-trained word embeddings in a Keras model

훈련 데이터가 적다면 케라스의 embedding() 을 사용하는 것보다 다른 텍스트 데이터로 사전 훈련되어 있는 임베딩 벡터를 불러오는 것이 나은 선택이다.

keras 블로그 주소 : https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html 
위 페이지 예제를 실습 해보자

GloVe word embeddings


We will be using GloVe embeddings, which you can read about here. GloVe stands for "Global Vectors for Word Representation". It's a somewhat popular embedding technique based on factorizing a matrix of word co-occurence statistics.

Specifically, we will use the 100-dimensional GloVe embeddings of 400k words computed on a 2014 dump of English Wikipedia. You can download them here (warning: following this link will start a 822MB download).

# ELMO


ELMo는 Embeddings from Language Model의 약자입니다. 해석하면 '언어 모델로 하는 임베딩'입니다. ELMo의 가장 큰 특징은 사전 훈련된 언어 모델(Pre-trained language model)을 사용한다는 점입니다. 이는 ELMo의 이름에 LM이 들어간 이유입니다.

Bank라는 단어를 생각해봅시다. Bank Account(은행 계좌)와 River Bank(강둑)에서의 Bank는 전혀 다른 의미를 가지는데, Word2Vec이나 Glove 등으로 표현된 임베딩 벡터들은 이를 제대로 반영하지 못한다는 단점이 있습니다. 예를 들어서 Word2Vec이나 Glove 등의 임베딩 방법론으로 Bank란 단어를 [0.2 0.8 -1.2]라는 임베딩 벡터로 임베딩하였다고 하면, 이 단어는 Bank Account(은행 계좌)와 River Bank(강둑)에서의 Bank는 전혀 다른 의미임에도 불구하고 두 가지 상황 모두에서 [0.2 0.8 -1.2]의 벡터가 사용됩니다.

그렇다면 같은 표기의 단어라도 문맥에 따라서 다르게 워드 임베딩을 할 수 있으면 자연어 처리의 성능이 더 올라가지 않을까요? 단어를 임베딩하기 전에 전체 문장을 고려해서 임베딩을 하겠다는 것이죠. 그래서 탄생한 것이 문맥을 반영한 워드 임베딩(Contextualized Word Embedding)입니다.

## ELMo표현을 사용해서 스팸 메일 분류하기

텐서플로우 허브로부터 다양하 사전 훈련된 모델(Pre-trained Model)들을 사용할 수 있다. 여기서는 사전훈련된 모델로부터 ELMo 표현을 사용해보는 정도로 예제를 진행해 보겠다, 시작전에 텐서플로우 허브를 인스톨 해야한다,

#pip install tensorflow-hub

In [2]:
!pip install tensorflow-hub

In [4]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K

sess = tf.Session()
K.set_session(sess)
#세션을 초기화,

Using TensorFlow backend.


In [5]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

In [6]:
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

기본적으로 필요한 것들을 임포트 하였습니다 이제 데이터를 불러오고, 5개만 출력해 보도록 하자

파일 다운로드 링크 : https://www.kaggle.com/uciml/sms-spam-collection-dataset

In [20]:
import pandas as pd
data = pd.read_csv('datasets/spam.csv',encoding = 'latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


위에서 필요한건 v2열과 v1열이다 , v1열은 숫자 레이블로 바꿔야할 필요가 있기 때문에 이를 각각 X_data와 Y_data로 저장한다

In [21]:
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])

In [22]:
y_data = list(data['v1'])
x_data = list(data['v2'])

v2 열을 X_data 에 저장합니다 v1 열에 있는 ham과 spam 레이블을 각각 숫자 0과 1로 바꾸고 y_data에 저장합니다. 정상적으로 저장되었는지 이를 각각 5개만 출력해보자 

In [23]:
x_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [25]:
print(y_data[:5])

[0, 0, 1, 0, 0]


In [27]:
#훈련 데이터와 테스트 데이터를 8:2 비율로 분할한다
print(len(x_data))
n_of_train = int(len(x_data) * 0.8)
n_of_test = int(len(x_data) - n_of_train)

print(n_of_train)
print(n_of_test)

5572
4457
1115


4457개의 훈련데이터 1115개의 테스트 데이터의 양으로 하여 데이터를 분할 하였습니다.

In [29]:
import numpy as np
x_train = np.asarray(x_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
x_test = np.asarray(x_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

이제 ELMo 와 설계한 모델을 연결하는 작업들을 진행해보자, ELMo는 텐서플로우 허브로부터 가져온 것이기 떄문에 케라스에서 사용하기 위해서는 케라스에서 사요할 수 있도록 변환해주는 작업들이 필요하다.

In [31]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

In [32]:
# 모델을 설계 하자
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0913 22:06:09.920939 19184 deprecation_wrapper.py:119] From D:\user\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0913 22:06:09.926126 19184 deprecation_wrapper.py:119] From D:\user\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0913 22:06:14.221179 19184 deprecation_wrapper.py:119] From D:\user\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0913 22:06:14.304988 19184 deprecation_wrapper.py:119] From D:\user\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0913 22:06:14.345767 19184 deprecation_wrapper.py:119] From D:\user\envs\tensorflow

In [35]:
history = model.fit(x_train, y_train, epochs=1, batch_size=60)

Epoch 1/1
4457/4457 [==============================] - 728s 163ms/step - loss: 0.0769 - acc: 0.9744


In [37]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

1115/1115 [==============================] - 156s 140ms/step

 테스트 정확도: 0.9731


# BPE & WordPiece & SentencePiece 임베딩은 다음에 알아보도록 하겠습니다